# AVG files

MTEdit processing code developed by Zonge International outputs .avg files which have minimal metadata and contain statistical estimates of the impedance, tipper and data errors.  

In [1]:
from mt_metadata.transfer_functions.io.zonge import ZongeMTAvg
from mt_metadata.transfer_functions.core import TF
from mt_metadata import TF_AVG

## AVG file structure

AVG files are not common, here is the basic structure.  Some metadata about how the station was processed, orientation of setup, and location, and then the statistical estimates with some parameters on coherency and Fourier Coefficients used.  

In [2]:
with open(TF_AVG, "r") as fid:
    lines = fid.readlines()

print("".join(lines[0:18]))

 
$Survey.Type=NSAMT
$Survey.Array=Tensor
$Tx.Type=Natural 
$MTEdit:Version=3.10m applied 2021/01/27
$MTEdit:Auto.PhaseFlip=Yes
$MTEdit:PhaseSlope.Smooth=Robust
$MTEdit:PhaseSlope.toZmag=No
$MTEdit:DPlus.Use=No
$Rx.GdpStn= 24
$Rx.Length=100 M
$Rx.HPR=0,0,180
$GPS.Lat=32.83331167
$GPS.Lon=-107.08305667
$Unit.Length=m
Skp,Freq,      E.mag,      B.mag,      Z.mag,      Z.phz,   ARes.mag,   ARes.%err,Z.perr,  Coher,   FC.NUse,FC.NTry
$Rx.Cmp = Zxx
2,  2.3438E-2, 7.8081E-01, 2.9611E-01, 1.3930E+00, -2228.9, 1.6558E+01, 98.4,     514.3,   0.971,   8,      16



In [3]:
tf_object = TF(fn=TF_AVG)
avg_object = ZongeMTAvg(fn=TF_AVG)

In [4]:
tf_object.station_metadata

{
    "station": {
        "acquired_by.author": null,
        "channels_recorded": [
            "hx",
            "ex",
            "ey",
            "hy"
        ],
        "data_type": "nsamt",
        "geographic_name": null,
        "id": "24",
        "location.datum": null,
        "location.declination.epoch": null,
        "location.declination.model": "EMAG2",
        "location.declination.value": 0.0,
        "location.elevation": 0.0,
        "location.latitude": 32.83331167,
        "location.longitude": -107.08305667,
        "orientation.method": null,
        "orientation.reference_frame": "geographic",
        "provenance.creation_time": "1980-01-01T00:00:00+00:00",
        "provenance.software.author": null,
        "provenance.software.last_updated": "1980-01-01T00:00:00+00:00",
        "provenance.software.name": null,
        "provenance.software.version": null,
        "provenance.submitter.author": null,
        "provenance.submitter.email": null,
        "prove

## Statistical Estimates

The data are stored as magnitude and phase of the impedance which are converted to real and imaginary values.

In [5]:
print(avg_object.z[0])
tf_object.impedance[0]

[[-0.85198341-1.1020768j  -1.76236149-1.89288924j]
 [-0.17290554-0.45221147j -0.16184071-0.1545914j ]]


<xarray.DataArray 'impedance' (output: 2, input: 2)>
array([[-0.85198341-1.1020768j , -1.76236149-1.89288924j],
       [-0.17290554-0.45221147j, -0.16184071-0.1545914j ]])
Coordinates:
  * output   (output) <U2 'ex' 'ey'
  * input    (input) <U2 'hx' 'hy'
    period   float64 42.67
Attributes:
    survey:             None
    project:            None
    id:                 24
    name:               None
    latitude:           32.83331167
    longitude:          -107.08305667
    elevation:          0.0
    declination:        0.0
    datum:              None
    acquired_by:        None
    start:              1980-01-01T00:00:00+00:00
    end:                1980-01-01T00:00:00+00:00
    runs_processed:     ['001']
    coordinate_system:  geographic

In [6]:
print(avg_object.z_err[0])
tf_object.impedance_error[0]

[[0.492  0.3165]
 [0.1025 0.278 ]]


<xarray.DataArray 'impedance_error' (output: 2, input: 2)>
array([[0.492 , 0.3165],
       [0.1025, 0.278 ]])
Coordinates:
  * output   (output) <U2 'ex' 'ey'
  * input    (input) <U2 'hx' 'hy'
    period   float64 42.67
Attributes:
    survey:             None
    project:            None
    id:                 24
    name:               None
    latitude:           32.83331167
    longitude:          -107.08305667
    elevation:          0.0
    declination:        0.0
    datum:              None
    acquired_by:        None
    start:              1980-01-01T00:00:00+00:00
    end:                1980-01-01T00:00:00+00:00
    runs_processed:     ['001']
    coordinate_system:  geographic